<a href="https://colab.research.google.com/github/mssamoilenko/AnlDataPract/blob/main/PraktAnl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практична робота №3
## Тема: класифікація за допомогою логістичної
## регресії та наївного Баєса.
## Мета: закріплення навичок роботи із категорійними даними, їх по
## передньої обробки та класифікація даних.
## Завдання
## 1.    Використати дані відповідно до власного варіанта (в мене це варіант №2). Вивести таблицю за допомогою Pandas.
## 2.   Визначити числові, категорійні впорядковані, категорійні невпорядковані
## змінні. Для кожної категорійної змінної написати код, який виводить всі
## унікальні значення, котрі вона набуває в наборі даних.
## 3.   Закодувати категорійні впорядковані числами 0, 1, 2… (або іншими
## числами аналогічним чином), до невпорядкованих застосувати опорне
## кодування.
## 4.   Застосувати методи логістичної регресії та наївного Баєса до оброблених
## даних для передбачення змінної «Статус проєкта». На крос-валідації обрати
## кращу модель.
## Варіант роботи № 2
## Хід роботи

## 1.   Виводжу таблицю з даними відповідно моєму варіанту.







In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
df = pd.read_csv("2.csv")
df

,Потужність (л/г),Тара,Якість,Структура,Відстань від джерела (км),Матеріал,Вартість (грн),Статус проєкта
0,38.33,Особливе замовлення,Низька,Ромбовидна,2479.88,Скло,954.65,Заблоковано
1,76.99,Особливе замовлення,Дуже низька,Фрактальна,1825.69,Залізо,1216.99,Узгоджено
2,48.72,Каністра,Дуже низька,Решітка,2034.32,Залізо,861.93,Узгоджено
3,63.23,Своя,Низька,Ромбовидна,2518.65,Поліпропілен,1055.64,Заблоковано
4,24.63,Бочка,Дуже низька,Фрактальна,2661.19,Скло,1063.26,Заблоковано
...,...,...,...,...,...,...,...,...
4995,37.36,Бочка,Висока,Решітка,2462.35,Льон,911.51,Заблоковано
4996,41.68,Бочка,Висока,Фрактальна,2774.46,Залізо,1136.20,Заблоковано
4997,51.92,Каністра,Середня,Ромбовидна,2482.63,Льон,1056.54,Узгоджено
4998,49.98,Каністра,Середня,Фрактальна,1768.00,Поліпропілен,1134.95,Узгоджено


## 2. Визначаю числові, категорійні впорядковані, категорійні невпорядковані змінні. Для кожної категорійної змінної пишу код, який виводить всі унікальні значення, котрі вона набуває в наборі даних.

In [64]:
# 1. Числові змінні
numerical_vars = ['Потужність (л/г)', 'Відстань від джерела (км)', 'Вартість (грн)']

In [65]:
# 2. Категорійні впорядковані змінні
ordinal_vars = ['Якість']

In [66]:
# 3. Категорійні невпорядковані змінні
categorical_vars = ['Тара', 'Структура', 'Матеріал', 'Статус проєкта']

In [67]:
df['Якість'].unique()

array(['Низька', 'Дуже низька', 'Середня', 'Висока'], dtype=object)

In [68]:
df['Тара'].unique()

array(['Особливе замовлення', 'Каністра', 'Своя', 'Бочка'], dtype=object)

In [69]:
df['Структура'].unique()

array(['Ромбовидна', 'Фрактальна', 'Решітка'], dtype=object)

In [70]:
df['Матеріал'].unique()

array(['Скло', 'Залізо', 'Поліпропілен', 'Льон'], dtype=object)

In [71]:
df['Статус проєкта'].unique()

array(['Заблоковано', 'Узгоджено'], dtype=object)

## 3. Я здійснила кодування категорійних ознак: впорядковані змінні закодовано числовими значеннями (0, 1, 2, 3 відповідно до рівня ознаки), а до невпорядкованих застосовано опорне (one-hot) кодування.

In [72]:
# Копіюємо набір
df_encoded = df.copy()

In [73]:
quality_order = {'Дуже низька': 0, 'Низька': 1, 'Середня': 2, 'Висока': 3}
df_encoded['Якість'] = df_encoded['Якість'].map(quality_order)

In [74]:
# One-hot encoding для решти змінних
df_encoded = pd.get_dummies(
    df_encoded,
    columns=['Тара', 'Структура', 'Матеріал', 'Статус проєкта'],
    drop_first=True,
    dtype=int
)

In [75]:
df_encoded.head()

,Потужність (л/г),Якість,Відстань від джерела (км),Вартість (грн),Тара_Каністра,Тара_Особливе замовлення,Тара_Своя,Структура_Ромбовидна,Структура_Фрактальна,Матеріал_Льон,Матеріал_Поліпропілен,Матеріал_Скло,Статус проєкта_Узгоджено
0,38.33,1,2479.88,954.65,0,1,0,1,0,0,0,1,0
1,76.99,0,1825.69,1216.99,0,1,0,0,1,0,0,0,1
2,48.72,0,2034.32,861.93,1,0,0,0,0,0,0,0,1
3,63.23,1,2518.65,1055.64,0,0,1,1,0,0,1,0,0
4,24.63,0,2661.19,1063.26,0,0,0,0,1,0,0,1,0


## 4.  Застосовую методи логістичної регресії та наївного Баєса до оброблених даних для передбачення змінної «Статус проєкта». На крос-валідації обераю кращу модель.

In [76]:
# 2. Визначаю ознаки (X) та цільову змінну (y)
X = df_encoded.drop('Статус проєкта_Узгоджено', axis=1)
y = df_encoded['Статус проєкта_Узгоджено']

In [77]:
# 3. Масштабую числові ознаки (для логістичної регресії це важливо)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [78]:
# 4. Створюю моделі
log_reg = LogisticRegression(max_iter=1000, random_state=42)
nb = GaussianNB()

In [79]:
# 5. Проводжу крос-валідацію для обох моделей (5 фолдів)
cv_log = cross_val_score(log_reg, X_scaled, y, cv=5, scoring='accuracy')
cv_nb = cross_val_score(nb, X_scaled, y, cv=5, scoring='accuracy')

In [80]:
# 6. Виводжу середню точність для кожної моделі
print("Середня точність логістичної регресії:", cv_log.mean())
print("Середня точність наївного Байєса:", cv_nb.mean())

Середня точність логістичної регресії: 0.9954000000000001
Середня точність наївного Байєса: 0.9454


In [81]:
# 7. Визначаю, яка модель краща
if cv_log.mean() > cv_nb.mean():
    print("Краща модель: Логістична регресія")
else:
    print("Краща модель: Наївний Байєс")

Краща модель: Логістична регресія
